In [1]:
"""
格式：
    rgb/存放有图片
    pose/存放有位姿
    intrinsic存放内参
"""

import os
import codecs
import shutil
import linecache
import pandas as pd
import numpy as np


import torch
import torch.utils.data as Data


torch.set_printoptions(precision=15)
# torch.set_default_tensor_type(torch.DoubleTensor)
"""计算四元数转旋转矩阵"""
def q2rot(qMat):
# qMat格式  qw qx qy qz
    w = qMat[0]
    x = qMat[1]
    y = qMat[2]
    z = qMat[3]
#     rotMat = [[1-2*y**2-2*z**2,2*x*y+2*w*z,2*x*z-2*w*y],
#               [2*x*y-2*w*z,1-2*x**2-2*z**2,2*y*z+2*w*x],
#               [2*x*z+2*w*y,2*y*z-2*w*x+1-2*x**2-2*y**2]]
    
    
    rotMat = np.mat([[1-2*y**2-2*z**2,2*x*y-2*w*z,2*x*z+2*w*y],
              [2*x*y+2*w*z,1-2*x**2-2*z**2,2*y*z-2*w*x],
              [2*x*z-2*w*y,2*y*z+2*w*x,1-2*x**2-2*y**2]])
    return rotMat

"""通过四元数，平移矩阵获得奇次矩阵"""
def q2homoRot(qMat,qTrans):
# qMat格式  qw qx qy qz
# qTrans格式 x y z
    qw = qMat[0]
    qx = qMat[1]
    qy = qMat[2]
    qz = qMat[3]
    x = qTrans[0]
    y = qTrans[1]
    z = qTrans[2]
    
    
#     rotMat = [[1-2*y**2-2*z**2,2*x*y+2*w*z,2*x*z-2*w*y],
#               [2*x*y-2*w*z,1-2*x**2-2*z**2,2*y*z+2*w*x],
#               [2*x*z+2*w*y,2*y*z-2*w*x+1-2*x**2-2*y**2]]
    
    
    rotMat = np.mat([
              [1-2*qy**2-2*qz**2,  2*qx*qy-2*qw*qz,   2*qx*qz+2*qw*qy,  x],
              [2*qx*qy+2*qw*qz,    1-2*qx**2-2*qz**2, 2*qy*qz-2*qw*qx,  y],
              [2*qx*qz-2*qw*qy,    2*qy*qz+2*qw*qx,   1-2*qx**2-2*qy**2,z],
              [0,                   0,                  0,                 1]
    ])
    return rotMat



In [2]:
model_name = "horse43/horse43grid"

bPath = "/home/vr717/Documents/qys/code/pointnerf/data_src/scannet/scans"
dirPath = bPath + "/" + model_name

video_path = dirPath+ "/video/video.mp4"
save_path = dirPath + "/images"
filePath = dirPath + "/sparse/0/images.txt"
imgPath = dirPath + "/images"
posePath = dirPath + "/pose"

print(video_path,save_path,filePath,imgPath,posePath)

/home/vr717/Documents/qys/code/pointnerf/data_src/scannet/scans/horse43/horse43grid/video/video.mp4 /home/vr717/Documents/qys/code/pointnerf/data_src/scannet/scans/horse43/horse43grid/images /home/vr717/Documents/qys/code/pointnerf/data_src/scannet/scans/horse43/horse43grid/sparse/0/images.txt /home/vr717/Documents/qys/code/pointnerf/data_src/scannet/scans/horse43/horse43grid/images /home/vr717/Documents/qys/code/pointnerf/data_src/scannet/scans/horse43/horse43grid/pose


In [3]:
"""
对视频抽帧
"""

import os.path
import time
import cv2

os.makedirs(save_path, exist_ok=True)
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
print('FPS:{:.2f}'.format(fps))
rate = cap.get(5)
frame_num = cap.get(7)
duration = frame_num/rate
print('video total time:{:.2f}s'.format(duration))
 
# width, height = 1920, 1080
# interval = int(fps) * 4
interval = 3
process_num = frame_num // interval
print('process frame:{:.0f}'.format(process_num))
 
cnt = 0
num = 0
 
t0 = time.time()
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        cnt += 1
        if cnt % interval == 0:
            num += 1
            # frame = cv.resize(frame, (width, height))
            cv2.imwrite(save_path + "/%d.jpg" % (num-1), frame)
            remain_frame = process_num - num
            t1 = time.time() - t0
            t0 = time.time()
#             print("Processing %07d.jpg, remain frame: %d, remain time: %.2fs" % (num, remain_frame, remain_frame * t1))
            print("Processing %d.jpg, remain frame: %d, remain time: %.2fs" % (num-1, remain_frame, remain_frame * t1))
    else:
        break
    if cv2.waitKey(1) & 0xff == 27:
        break
 
cap.release()
cv2.destroyAllWindows()
print("done")




FPS:30.00
video total time:25.37s
process frame:253
Processing 0.jpg, remain frame: 252, remain time: 30.53s
Processing 1.jpg, remain frame: 251, remain time: 4.46s
Processing 2.jpg, remain frame: 250, remain time: 4.00s
Processing 3.jpg, remain frame: 249, remain time: 4.04s
Processing 4.jpg, remain frame: 248, remain time: 4.16s
Processing 5.jpg, remain frame: 247, remain time: 4.13s
Processing 6.jpg, remain frame: 246, remain time: 4.27s
Processing 7.jpg, remain frame: 245, remain time: 4.44s
Processing 8.jpg, remain frame: 244, remain time: 6.12s
Processing 9.jpg, remain frame: 243, remain time: 4.36s
Processing 10.jpg, remain frame: 242, remain time: 4.29s
Processing 11.jpg, remain frame: 241, remain time: 4.19s
Processing 12.jpg, remain frame: 240, remain time: 4.33s
Processing 13.jpg, remain frame: 239, remain time: 4.12s
Processing 14.jpg, remain frame: 238, remain time: 4.19s
Processing 15.jpg, remain frame: 237, remain time: 4.09s
Processing 16.jpg, remain frame: 236, remain 

Processing 143.jpg, remain frame: 109, remain time: 1.87s
Processing 144.jpg, remain frame: 108, remain time: 1.77s
Processing 145.jpg, remain frame: 107, remain time: 1.82s
Processing 146.jpg, remain frame: 106, remain time: 1.76s
Processing 147.jpg, remain frame: 105, remain time: 1.70s
Processing 148.jpg, remain frame: 104, remain time: 1.62s
Processing 149.jpg, remain frame: 103, remain time: 1.62s
Processing 150.jpg, remain frame: 102, remain time: 1.67s
Processing 151.jpg, remain frame: 101, remain time: 1.66s
Processing 152.jpg, remain frame: 100, remain time: 1.68s
Processing 153.jpg, remain frame: 99, remain time: 1.57s
Processing 154.jpg, remain frame: 98, remain time: 1.52s
Processing 155.jpg, remain frame: 97, remain time: 1.47s
Processing 156.jpg, remain frame: 96, remain time: 1.58s
Processing 157.jpg, remain frame: 95, remain time: 1.56s
Processing 158.jpg, remain frame: 94, remain time: 1.54s
Processing 159.jpg, remain frame: 93, remain time: 1.54s
Processing 160.jpg, r

In [4]:
import os
import logging
import subprocess

log = logging.getLogger("Core.Analysis.Processing")

INTERPRETER = "/usr/bin/python"


if not os.path.exists(INTERPRETER):
  log.error("Cannot find INTERPRETER at path \"%s\"." % INTERPRETER)

processor = "/home/vr717/Documents/qys/code/LLFF/imgs2poses.py"

pargs = [INTERPRETER, processor]
pargs.extend([""+dirPath])
p = subprocess.Popen(pargs, shell=False, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

while p.poll() is None:
    line = p.stdout.readline()
    line = line.strip()
    if line:
        print('Subprogram output: [{}]'.format(line))
    if p.returncode == 0:
        print('Subprogram success')
    else:
        print('Subprogram failed')


Subprogram output: [b'Need to run COLMAP']
Subprogram failed
Subprogram output: [b'Features extracted']
Subprogram failed
Subprogram output: [b'Features matched']
Subprogram failed
Subprogram output: [b'Sparse map created']
Subprogram failed
Subprogram output: [b'Finished running COLMAP, see /home/vr717/Documents/qys/code/pointnerf/data_src/scannet/scans/horse43/horse43grid/colmap_output.txt for logs']
Subprogram failed
Subprogram output: [b'Post-colmap']
Subprogram failed
Subprogram output: [b'Cameras 5']
Subprogram failed
Subprogram output: [b'Images # 253']
Subprogram failed
Subprogram output: [b'Points (62806, 3) Visibility (62806, 253)']
Subprogram failed
Subprogram output: [b'Depth stats 0.06575675969710001 177.53321976851043 6.201408477031065']
Subprogram failed
Subprogram output: [b'Done with imgs2poses']
Subprogram failed
Subprogram failed


In [6]:

# 修改图片名称
imgListOri = os.listdir(imgPath)

counter = 0
trainCounter = 0
valCounter = 0

for img in imgListOri:
    
        
    newName =str(str(counter)+".jpg")
    counter = counter+1
    src =  os.path.join(imgPath,img)
    dst = os.path.join(imgPath,newName)
    try:
        os.rename(src,dst)

    except:
        print("error")


In [3]:
"""
读取colmap 进行旋转矩阵 输出最大值最小值
""" 



imgNum = len(os.listdir(imgPath))
print(imgNum)

df = pd.read_table(filePath)

rawDf = []
nameList = []
Df = []
qMat = []

for i in range(imgNum):
    line = df.loc[i*2 + 3].values[0]
    rawDf.append(line)
print(len(rawDf))


postList = [] #存有每个图片的位姿信息

xList = []
yList = []
zList = []

for i in rawDf:
    line = i.split(' ')
    Df.append(line)
    nameList.append(line[-1])
        
    qMat = []
    qTrans = []
    for i in line[1:5]:
        qMat.append(float(i))
    for i in line[5:8]:
        qTrans.append(float(i))
    
    w2per = q2homoRot(qMat,qTrans)
    #转置 计算成相机到世界的

    testMat = torch.tensor(w2per)

    tra = testMat[:3,:3]
    pos = testMat[:3,3].reshape(3,1)
    
    invMat = torch.transpose(tra,0,1)
#     print((-1*invMat).shape)
#     print((-1*invMat))
    
#     print(pos.shape)
#     print(pos)

# # 左手席右手系转换
#     le2ri = torch.tensor([[1,0,0],[0,1,0],[0,0,-1]])
#     invMat = torch.mm(le2ri.to(torch.float64),invMat)
    
    finalPos =torch.mm((-1*invMat),pos.to(torch.float64))
    print("finalpos",finalPos)
    
    
    
    tem = torch.cat((invMat,finalPos),1)
    finalMat = torch.cat((tem,torch.tensor([[0,0,0,1]])),0)
#     print(finalMat)
    postList.append(finalMat)
    
    
    

os.makedirs(posePath, exist_ok=True)
for ind in range(len(postList)):
    txtName = nameList[ind].split('.')[0] + ".txt"
    with codecs.open( os.path.join(posePath,txtName), 'w','gb18030') as f:
        temList = postList[ind].tolist()
        xList.append(temList[0][-1])
        yList.append(temList[1][-1])
        zList.append(temList[2][-1])
        for item in temList:
#             print(item)
            temLine  = ' '.join(str(i) for i in item)
            temLine = temLine + "\n"
            f.write(temLine)


bbox = [min(xList),min(yList),min(zList),max(xList),max(yList),max(zList)]
print(bbox)      

253
253
finalpos tensor([[-1.922134973334500],
        [-2.391350994391249],
        [ 2.045540606269775]], dtype=torch.float64)
finalpos tensor([[-1.666048576571156],
        [-2.498591579754422],
        [ 2.137960977928746]], dtype=torch.float64)
finalpos tensor([[-1.440469382881037],
        [-2.589774282117285],
        [ 2.240147701195320]], dtype=torch.float64)
finalpos tensor([[-1.238312915261740],
        [-2.676909600844282],
        [ 2.347427782846362]], dtype=torch.float64)
finalpos tensor([[-1.015115416889069],
        [-2.747691663694833],
        [ 2.460953137972321]], dtype=torch.float64)
finalpos tensor([[-0.751350658392136],
        [-2.850385462492190],
        [ 2.590496043045035]], dtype=torch.float64)
finalpos tensor([[-0.509536366107710],
        [-2.906780718571807],
        [ 2.734843238421899]], dtype=torch.float64)
finalpos tensor([[-0.234106021622626],
        [-2.970345757194210],
        [ 2.850046550825474]], dtype=torch.float64)
finalpos tensor([[ 0.067